In [302]:
import os
import sys
import gmsh
import numpy as np
from functools import partial

In [144]:
gmsh.initialize()

In [145]:
f_mesh = '../data/simnibs_output/sub-CMH090.msh'

In [146]:
#Open with gmsh, then 
gmsh.open(f_mesh)

In [147]:
gmsh.model.getEntities()

[(2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7)]

#### GMSH file anatomy
<code>gmsh.model.mesh.getElements</code> yields a list of elements with elem_coords, which correspond to node_tags. 
These node_tags are derived from the following sources:

- nodes belonging to the tetrahedral volume 3D mesh
- nodes belonging to the grey matter boundary mesh
- nodes belonging to the white matter boundary mesh

In [226]:
# Get grey matter volume
tet_gm = (3,2)
tet_node_tag, tet_node_coord, tet_node_param = gmsh.model.mesh.getNodes(tet_gm[0],tet_gm[1])
tet_elem_tag, tet_elem_coord, tet_elem_param = gmsh.model.mesh.getElements(tet_gm[0],tet_gm[1])

In [227]:
# Get grey matter boundary surface
surf_gm = (2,2)
gm_node_tag, gm_node_coord, gm_node_param = gmsh.model.mesh.getNodes(surf_gm[0],surf_gm[1])
_, gm_elem_coord, gm_elem_param = gmsh.model.mesh.getElements(surf_gm[0],surf_gm[1])

In [228]:
# Get white matter boundary surface
surf_wm = (2,1)
wm_node_tag, wm_node_coord, wm_node_param = gmsh.model.mesh.getNodes(surf_wm[0], surf_wm[1])
_, wm_elem_coord, wm_elem_param = gmsh.model.mesh.getElements(surf_wm[0], surf_wm[1])

In [236]:
# Convert to set of nodes belonging to a single triangle
tet_node_list = tet_elem_param[0].reshape((tet_elem_param[0].shape[0]//4, 4))

In [280]:
tet_coord_map = { n : tet_node_coord[3*i:(3*i)+3] for i,n in enumerate(tet_node_tag) }
gm_coord_map = { n : gm_node_coord[3*i:(3*i)+3] for i,n in enumerate(gm_node_tag) }
wm_coord_map = { n : wm_node_coord[3*i:(3*i)+3] for i,n in enumerate(wm_node_tag) }

# Bring together dictionaries
node_2_coord = {}
node_2_coord.update(tet_coord_map)
node_2_coord.update(gm_coord_map)
node_2_coord.update(wm_coord_map)

In [303]:
# For each element in the list, loop over and compute barycentre
def compute_barycentre(x,mapping):
    
    #Map node --> coord set
    c1 = mapping[x[0]]
    c2 = mapping[x[1]]
    c3 = mapping[x[2]]
    c4 = mapping[x[3]]
    
    #Compute barycentre
    return np.divide(c1+c2+c3+c4,4)
barycentre = partial(compute_barycentre,mapping=node_2_coord)

In [309]:
# Compute list of barycentres for each of the tetrahedral elements
bary_list = np.apply_along_axis(barycentre,1,tet_node_list)

In [ ]:
# See if tetrahedral coordinates can be mapped to volume space? 